#### (1) LCEL 인터페이스

* 사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현 **`-> 일단 의존성 패키지 충돌로 사용 X`**

* `Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있음 

* **표준 인터페이스**로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 함

    * [`stream`](#stream): 응답의 청크를 스트리밍
    * [`invoke`](#invoke): 입력에 대해 체인을 호출
    * [`batch`](#batch): 입력 목록에 대해 체인을 호출

* **비동기 메소드**

    * [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍
    * [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출
    * [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출
    * [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍

#### (2) 표준 인터페이스

In [ ]:
# 기본 모듈 임포트
import os
import asyncio
from dotenv import load_dotenv

# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()                   # true

In [ ]:
# 환경 변수 확인하기

# 마스킹 처리 함수 정의
def mask_key(key: str, visible_count: int = 2) -> str:
    if not key or len(key) <= visible_count:
        return '*' * len(key)
    return key[:visible_count] + '*' * (len(key) - visible_count)

# 환경변수 불러오기
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY 환경 변수가 설정되지 않았습니다.")

# 마스킹된 형태로 출력
print(f"GOOGLE_API_KEY: {mask_key(api_key)}")           # GOOGLE_API_KEY: AI*************************************

In [ ]:
# LangSmith 추적 설정 (https://smith.langchain.com)

"""
- !pip install -qU langsmith
- !pip install -qU langchain-teddynote
    -> 제미나이와 poetry와의 의존성 충돌로 langchain_teddy 설치 X 
    -> langsmith로 진행
"""
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable         # @traceable 데코레이터 사용 시

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 셀 출력
    * --- LangSmith 환경 변수 확인 ---
    * ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    * ✅ LangSmith 프로젝트: 'L***************'
    * ✅ LangSmith API Key: 설정됨
    *   -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.

In [ ]:
# LangChain 및 Google GenAI 모델 관련 모듈 임포트
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI           # Google GenAI 임포트


print("\n--- LangChain 체인 설정 ---")

template = "{topic}에 대해 3문장으로 설명해줘."                           # 템플릿 정의

prompt = PromptTemplate.from_template(template)                     # 프롬프트를 프롬프트템플릿 객체로 생성

try:
    model = ChatGoogleGenerativeAI(                                 # 모델 호출
        model="gemini-1.5-flash",
        temperature=0.1,
    )
    print("✅ Google GenAI 모델 초기화 성공.")
except Exception as e:                                              # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    
# 출력 파서
output_parser = StrOutputParser()

# 체인 구성
chain = prompt | model | output_parser                              # 프롬프트, 모델, 출력 파서 연결 -> 체인 구성
print("✅ LangChain LCEL 체인 구성 완료.")

<small>

* 셀 출력
    * --- LangChain 체인 설정 ---
    * ✅ Google GenAI 모델 초기화 성공.
    * ✅ LangChain LCEL 체인 구성 완료.

---

#### (2) 표준 인터페이스

* **stream: 실시간 출력**

  * **실시간으로 조각조각 응답 받기**
    * Q. 언제 쓸까?
      * A1. 챗봇처럼 사용자와 실시간으로 대화하며 응답이 즉시 보이는 것이 중요한 경우 
      * A2. 긴 답변을 생성할 때 특히 유용
    * 특징_1: 응답이 생성되는 대로 작은 조각(토큰) 단위로 즉시 출력
    * 특징_2: 응답이 끝날 때까지 기다릴 필요가 없어서 사용자 경험이 더 부드러울 수 있음

  * `chain.stream` 메서드 -> 주어진 토픽에 대한 **데이터 스트림을 생성**
  * 이 스트림을 반복 -> **각 데이터의 내용(`content`)을 즉시 출력** 
      * `end=""` 인자 = 출력 후 줄바꿈을 하지 않도록 설정
      * `flush=True` 인자 = 출력 버퍼를 즉시 비우도록 함

In [ ]:
# 스트리밍 출력을 위한 헬퍼 함수 (03_파일 참고)
def stream_and_print_response(answer_iterator):
    for chunk in answer_iterator:
        print(chunk, end="", flush=True)
    print()                                         # 스트림 완료 후 줄바꿈

print("\n--- '멀티모달' 토픽 스트림 출력 ---")
try:
    answer_stream_multimodal = chain.stream({"topic": "멀티모달"})                 # chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성
    stream_and_print_response(answer_stream_multimodal)                          # 헬퍼 함수를 사용 -> 스트림된 데이터 출력
    print("✅ '멀티모달' 스트림 호출 성공.")
    
except Exception as e:
    print(f"❌ '멀티모달' 스트림 중 오류 발생: {e}")
    print("  -> 모델 응답, API 키, 네트워크 연결 등을 확인해 보세요.")

<small>

* 셀 출력

    * --- '멀티모달' 토픽 스트림 출력 ---
    * 멀티모달은 텍스트, 이미지, 오디오, 비디오 등 여러 유형의 데이터를 동시에 사용하는 기술입니다.  이를 통해 단일 모달만 사용할 때보다 더 풍부하고 정확한 정보를 얻을 수 있습니다.  예를 들어, 이미지와 텍스트를 함께 분석하여 이미지의 내용을 더 정확하게 이해하거나, 음성과 영상을 결합하여 더욱 자연스러운 대화형 시스템을 구축할 수 있습니다.

    * ✅ '멀티모달' 스트림 호출 성공.

In [ ]:
# 새 프롬프트 템플릿, 새 모델, 출력 파서 정의 및 체인 구성 

# 새 모델
try:
    model2 = ChatGoogleGenerativeAI(                                 # 모델 호출
        model="gemini-2.5-flash-lite",
        temperature=0.1,
    )
    print("✅ Google GenAI 모델 초기화 성공.")
    print("---")
except Exception as e:                                              # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    print("---")

# 기존 prompt 대신 새로운 끝말잇기 프롬프트 템플릿을 정의
template_word_game = """
당신은 끝말잇기 게임을 하는 인공지능입니다.
사용자가 시작 단어를 제시하면, 그 단어의 마지막 글자로 시작하는 2글자 단어를 다음으로 제시하고, 이 과정을 총 10번 반복합니다.
각각의 단어는 한 줄에 한 개씩 적어주세요.

예시:
시작 단어: '바다'
응답:
다면
면도
도마

시작 단어: '{start_word}'
응답:
"""

# 새로운 프롬프트 템플릿 객체 생성 (변수명을 prompt_word_game으로 변경)
prompt_word_game = PromptTemplate.from_template(template_word_game)

# 모델은 기존 model (gemini-1.5-flash)을 그대로 사용
# output_parser도 StrOutputParser를 그대로 사용

# 체인 구성: prompt_word_game | model | output_parser
# 변수명 변경: chain_word_game -> 혼동 방지
chain_word_game = prompt_word_game | model2 | StrOutputParser()

print("✅ 끝말잇기 체인 구성 완료.")
print("---")

# 스트리밍 출력을 위한 헬퍼 함수 (이전에 정의한 함수가 없으면 이 부분을 다시 추가해야 합니다)
def stream_and_print_response(answer_iterator):
    for chunk in answer_iterator:
        print(chunk, end="", flush=True)
    print() # 스트림 완료 후 줄바꿈


# stream 호출

print("\n--- 끝말잇기 스트림 출력 (시작 단어: '나무') ---")
try:
    # chain_word_game.stream을 사용하여 '나무'로 시작하는 끝말잇기 스트림 생성
    answer_stream_word_game = chain_word_game.stream({"start_word": "나무"})
    stream_and_print_response(answer_stream_word_game)
    print("✅ 끝말잇기 스트림 호출 성공.")
    print("---")
    
except Exception as e:
    print(f"❌ 끝말잇기 스트림 중 오류 발생: {e}")
    print("  -> 모델 응답, API 키, 네트워크 연결 등을 확인해 보세요.")
    print("---")

<small>

* 셀 출력
    * ✅ Google GenAI 모델 초기화 성공.
    * ---
    * ✅ 끝말잇기 체인 구성 완료.
    * ---

    * --- 끝말잇기 스트림 출력 (시작 단어: '나무') ---
    * 나무
    * 무궁
    * 궁합
    * 합격
    * 격투
    * 투명
    * 명예
    * 예술
    * 술래
    * 래일
    * ✅ 끝말잇기 스트림 호출 성공.
    * ---

* **invoke: 호출**
  
  * `invoke`메서드: 주제를 인자로 받아 해당 주제에 대한 처리를 수행
  
  * **한 번에 완결된 응답 받기**
      * Q. 언제 쓸까? 
        * A. **`질문에 대한 답이 한 번에 딱 나오면 되는 경우`** (예: "대한민국의 수도는?", "이 단어의 정의는?")

      * 특징: 응답이 완전히 생성될 때까지 기다렸다가, 결과가 한 텍스트 덩어리로 한 번에 출력

In [ ]:
# invoke 메서드를 사용한 토픽 invoke
print("\n--- 'ChatGPT' 토픽 invoke 호출 결과 ---")
try:
    response_chatgpt_invoke = chain.invoke({"topic": "ChatGPT"})    # chain.invoke 메서드 호출 -> 'ChatGPT'라는 주제로 딕셔너리 전달
    print(response_chatgpt_invoke)
    print("✅ 'ChatGPT' invoke 호출 성공.")
    
except Exception as e:
    print(f"❌ 'ChatGPT' invoke 호출 중 오류 발생: {e}")
    print("  -> 모델 응답, API 키, 네트워크 연결, 또는 프롬프트 내용을 확인해 보세요.")

<small>

* 셀 출력

    * --- 'ChatGPT' 토픽 invoke 호출 결과 ---
    * ChatGPT는 구글에서 개발한 대규모 언어 모델입니다. 방대한 양의 텍스트 데이터를 학습하여 사람과 유사한 텍스트를 생성하고, 질문에 답하고, 다양한 창작물을 만들 수 있습니다.  이는 대화형 AI로서 다양한 분야에서 활용되고 있으며, 지속적으로 발전하고 있습니다.
    * ✅ 'ChatGPT' invoke 호출 성공.

In [ ]:
# 새 프롬프트 템플릿, 새 모델, 출력 파서 정의 및 체인 구성 

# 새 모델
try:
    model2 = ChatGoogleGenerativeAI(                                 # 모델 호출
        model="gemini-2.5-flash-lite",
        temperature=0.1,
    )
    print("✅ Google GenAI 모델 초기화 성공.")
    print("---")
except Exception as e:                                              # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    print("---")

# 기존 prompt 대신 새로운 끝말잇기 프롬프트 템플릿을 정의
template_word_game = """
당신은 끝말잇기 게임을 하는 인공지능입니다.
사용자가 시작 단어를 제시하면, 그 단어의 마지막 글자로 시작하는 2글자 단어를 다음으로 제시하고, 이 과정을 총 10번 반복합니다.
각각의 단어는 한 줄에 한 개씩 적어주세요.

예시:
시작 단어: '바다'
응답:
다면
면도
도마

시작 단어: '{start_word}'
응답:
"""

# 새로운 프롬프트 템플릿 객체 생성 (변수명을 prompt_word_game으로 변경)
prompt_word_game = PromptTemplate.from_template(template_word_game)

# 모델은 기존 model (gemini-1.5-flash)을 그대로 사용
# output_parser도 StrOutputParser를 그대로 사용

# 체인 구성: prompt_word_game | model | output_parser
# 변수명 변경: chain_word_game -> 혼동 방지
chain_word_game = prompt_word_game | model2 | StrOutputParser()

print("✅ 끝말잇기 체인 구성 완료.")
print("---")

# 스트리밍 출력을 위한 헬퍼 함수 (이전에 정의한 함수가 없으면 이 부분을 다시 추가해야 합니다)
def stream_and_print_response(answer_iterator):
    for chunk in answer_iterator:
        print(chunk, end="", flush=True)
    print() # 스트림 완료 후 줄바꿈


# invoke 호출

print("\n--- 끝말잇기 invoke 호출 결과 (시작 단어: '라면') ---")
try:
    # chain_word_game.invoke를 호출하여 '라면'으로 시작하는 끝말잇기 결과를 한 번에 받습니다.
    response_word_game_invoke = chain_word_game.invoke({"start_word": "라면"})
    print(response_word_game_invoke)
    print("✅ 끝말잇기 invoke 호출 성공.")
except Exception as e:
    print(f"❌ 끝말잇기 invoke 호출 중 오류 발생: {e}")
    print("  -> 모델 응답, API 키, 네트워크 연결 등을 확인해 보세요.")

<small>

* 셀 출력

    * ✅ Google GenAI 모델 초기화 성공.
    * ---
    * ✅ 끝말잇기 체인 구성 완료.
    * ---

    * --- 끝말잇기 invoke 호출 결과 (시작 단어: '라면') ---
    * 면도
    * 도마
    * 마늘
    * 늘보
    * 보리
    * 리본
    * 본격
    * 격투
    * 투명
    * ✅ 끝말잇기 invoke 호출 성공.

* **`batch`: 배치(단위 실행)**
    * 함수 `chain.batch` = 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행
  
    * Q. 언제 쓸까?
      * A1. **여러 질문에 대한 답을 한 번에 받고 싶을 때** 
        * 예시: 여러 개의 키워드에 대한 설명을 동시에 얻거나, 여러 문장을 한꺼번에 번역하고 싶을 때 유용
       
      * A2. **API 호출 비용이나 속도를 최적화하고 싶을 때** 
        * 초당 요청 수 제한이 있는 API를 사용할 경우 `max_concurrency`  -> 한 번에 보내는 요청 수를 제한 = 요청 수 제한 = 오류 방지 가능
        * 예시: 개별 invoke를 여러 번 호출하는 것보다 batch 한 번으로 묶어서 호출하면 API 요청 수가 줄어들거나 내부적으로 병렬 처리가 되어 효율적
      
      * A3. **데이터 파이프라인에서 일괄 처리가 필요할 때** 
        * 예시: 대량의 데이터를 모델에 통과시켜야 할 경우 효과적
  
    * 특징
      * **입력 = 딕셔너리 리스트 형태**로 전달 -> 각 딕셔너리는 하나의 개별 입력
      * **출력 = 결과 리스트 형태**로 반환 -> **입력 리스트의 순서와 동일하게** 각 입력에 대한 결과가 나열
      * **모든 결과**가 생성될 때까지 기다린 후에 **한 번에 반환** = `innoke`

In [17]:
# 앞서 사용한 chain을 사용
# chain = prompt | model | StrOutputParser()

import time

print("\n--- batch: 여러 토픽 일괄 설명 요청 ---")
start_time = time.time()                                                    # 시작 시간 기록

try:
    # 주어진 토픽 리스트를 batch 처리하여 각 토픽에 대한 설명을 요청
    batch_results = chain.batch([
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"}
    ])
    
    end_time = time.time()                                                  # 종료 시간 기록
        
    # 결과는 리스트 형태로 반환
    for i, res in enumerate(batch_results):
        print(f"[{i+1}] {res}")
    print("✅ batch 호출 성공.")
    print("---")
    print(f"✅ abatch 호출 성공. 소요 시간: {end_time - start_time:.2f} 초")     # 소요 시간 출력
    print("---")

except Exception as e:
    print(f"❌ batch 호출 중 오류 발생: {e}")
    print("  -> 모델 응답, API 키, 네트워크 연결 등을 확인해 보세요.")
    print("---")


--- batch: 여러 토픽 일괄 설명 요청 ---
[1] ChatGPT는 구글에서 개발한 대규모 언어 모델입니다. 방대한 양의 텍스트 데이터를 학습하여 사람과 유사한 텍스트를 생성하고, 질문에 답하고, 다양한 창작물을 만들 수 있습니다.  이는 대화형 AI로서, 사용자와 자연스러운 대화를 나누는 데 활용됩니다.
[2] Instagram은 사진과 비디오 공유를 위한 소셜 네트워킹 서비스입니다. 사용자는 사진과 비디오에 필터를 적용하고, 해시태그를 사용하여 게시물을 분류하고, 다른 사용자를 팔로우하여 콘텐츠를 볼 수 있습니다. 전 세계 수억 명의 사용자가 사진과 비디오를 공유하고 소통하는 인기 플랫폼입니다.
[3] 멀티모달은 텍스트, 이미지, 오디오, 비디오 등 여러 유형의 데이터를 동시에 사용하는 기술입니다.  이를 통해 단일 모달만 사용할 때보다 더 풍부하고 정확한 정보를 얻을 수 있습니다.  예를 들어, 이미지와 텍스트를 함께 분석하여 이미지의 내용을 더 정확하게 이해하거나, 오디오와 비디오를 결합하여 더욱 몰입적인 경험을 제공할 수 있습니다.
✅ batch 호출 성공.
---
✅ abatch 호출 성공. 소요 시간: 2.50 초
---


<small>

* 
  * 셀 출력
      * --- batch: 여러 토픽 일괄 설명 요청 ---
      * [1] ChatGPT는 구글에서 개발한 대규모 언어 모델입니다. 방대한 양의 텍스트 데이터를 학습하여 사람과 유사한 텍스트를 생성하고, 질문에 답하고, 다양한 창작물을 만들 수 있습니다.  이는 대화형 AI로서, 사용자와 자연스러운 대화를 나누는 데 활용됩니다.
      * [2] Instagram은 사진과 비디오 공유를 위한 소셜 네트워킹 서비스입니다. 사용자는 사진과 비디오에 필터를 적용하고, 해시태그를 사용하여 게시물을 분류하고, 다른 사용자를 팔로우하여 콘텐츠를 볼 수 있습니다. 전 세계 수억 명의 사용자가 Instagram을 통해 소통하고, 콘텐츠를 공유하고, 브랜드와 연결됩니다.
      * [3] 멀티모달은 텍스트, 이미지, 오디오, 비디오 등 여러 유형의 데이터를 동시에 사용하는 기술입니다.  이를 통해 단일 모달만 사용할 때보다 더 풍부하고 정확한 정보를 얻을 수 있습니다.  예를 들어, 이미지와 텍스트를 함께 분석하여 이미지의 내용을 더욱 정확하게 이해하거나, 음성과 영상을 결합하여 더욱 자연스러운 대화형 시스템을 구축할 수 있습니다.
      * ✅ batch 호출 성공.
      * ---
      * ✅ abatch 호출 성공. 소요 시간: 2.50 초
      * ---

In [ ]:
# `max_concurrency` 사용 예시

chain_max_concurrency = prompt | model2 | StrOutputParser()         # gemini-2.5-flash-lite로 변경 -> 새 체인 구성

print("\n--- batch with max_concurrency: 동시 요청 수 제어 ---")
try:
    # 더 많은 토픽 리스트를 제공하고, max_concurrency를 2로 설정하여 동시 처리 수를 제한
    batch_concurrency_results = chain_max_concurrency.batch(
        [
            {"topic": "ChatGPT"},
            {"topic": "Instagram"},
            {"topic": "멀티모달"},
            {"topic": "프로그래밍"},
            {"topic": "머신러닝"},
            {"topic": "인공지능"}                                         # 새로운 토픽 추가
        ],
        config={"max_concurrency": 2}                                   # 동시에 2개의 요청만 처리하도록 설정
    )
    
    for i, res in enumerate(batch_concurrency_results):
        print(f"[{i+1}] {res}")
        # 배치 그룹(2개) 결과 출력 시 구분선 추가
        if (i + 1) % 2 == 0:                                            # 현재 인덱스(i)가 짝수 번째일 때 (0부터 시작하므로 +1)
            print("--- Batch 그룹 완료 ---")                              # 구분선 추가
    print("✅ max_concurrency를 사용한 batch 호출 성공.")
    print("---")
    
except Exception as e:
    print(f"❌ max_concurrency를 사용한 batch 호출 중 오류 발생: {e}")
    print("  -> 모델 응답, API 키, 네트워크 연결 등을 확인해 보세요.")
    print("---")

<small>

* 셀 출력
    * --- batch with max_concurrency: 동시 요청 수 제어 ---
    * [1] ChatGPT는 OpenAI에서 개발한 대규모 언어 모델로, 인간과 유사한 텍스트를 생성하고 다양한 질문에 답변할 수 있습니다. 방대한 양의 텍스트 데이터를 학습하여 문맥을 이해하고 창의적인 글쓰기, 번역, 요약 등 여러 작업을 수행할 수 있습니다. 대화형 인터페이스를 통해 사용자와 자연스럽게 소통하며 정보를 제공하고 아이디어를 발전시키는 데 도움을 줍니다.
    * [2] Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼입니다. 사용자는 필터를 적용하고 캡션을 추가하여 자신의 콘텐츠를 꾸밀 수 있으며, 다른 사용자의 게시물에 좋아요를 누르거나 댓글을 달아 소통할 수 있습니다. 또한, 스토리를 통해 24시간 동안만 유지되는 짧은 콘텐츠를 공유하거나, 라이브 방송을 통해 실시간으로 소통하는 것도 가능합니다.
    * --- Batch 그룹 완료 ---
    * [3] 멀티모달은 텍스트, 이미지, 오디오, 비디오 등 여러 종류의 데이터를 동시에 이해하고 처리하는 기술입니다. 이를 통해 인간처럼 다양한 정보를 종합적으로 파악하여 더욱 풍부하고 맥락에 맞는 상호작용이 가능해집니다. 예를 들어, 이미지와 함께 제공된 설명을 이해하거나, 음성 명령을 통해 비디오를 검색하는 등의 작업에 활용될 수 있습니다.
    * [4] 프로그래밍은 컴퓨터에게 특정 작업을 수행하도록 지시하는 명령어들의 집합을 만드는 과정입니다. 이러한 명령어들은 특정 언어(프로그래밍 언어)로 작성되며, 컴퓨터는 이 언어를 이해하고 실행하여 우리가 원하는 결과를 만들어냅니다. 프로그래밍을 통해 우리는 웹사이트, 애플리케이션, 게임 등 다양한 소프트웨어를 개발할 수 있습니다.
    * --- Batch 그룹 완료 ---
    * [5] 머신러닝은 컴퓨터가 명시적인 프로그래밍 없이도 데이터로부터 학습하고 예측하거나 의사결정을 내릴 수 있도록 하는 인공지능의 한 분야입니다. 알고리즘은 대규모 데이터셋을 분석하여 패턴을 파악하고, 이를 통해 새로운 데이터에 대한 예측 모델을 구축합니다. 이러한 학습 과정을 통해 머신러닝은 이미지 인식, 자연어 처리, 추천 시스템 등 다양한 분야에서 활용됩니다.
    * [6] 인공지능(AI)은 인간의 학습 능력, 추론 능력, 지각 능력 등을 컴퓨터 프로그램으로 실현한 기술입니다. AI는 방대한 데이터를 분석하고 패턴을 학습하여 스스로 의사결정을 내리거나 문제를 해결할 수 있습니다. 이러한 AI 기술은 의료, 금융, 교육 등 다양한 분야에서 혁신을 이끌고 있으며, 앞으로 더욱 발전하여 우리 삶에 큰 영향을 미칠 것으로 기대됩니다.
    * --- Batch 그룹 완료 ---
    * ✅ max_concurrency를 사용한 batch 호출 성공.
    * ---

#### (3) 비동기 메소드

* 동기와 비동기

    * **동기(`Synchronous`)**: 하나의 작업이 완료될 때까지 다음 작업이 대기하는 방식
      * 예시: 주방장이 한 음식 조리 완료 후 다음 음식 시작

    * **비동기(`Asynchronous`)**
      * 하나의 작업이 시작되면, 그 작업이 완료되기를 기다리지 않고 바로 다음 작업을 시작하는 방식
      * 이전 작업은 백그라운드에서 진행되고, 완료되면 결과를 알려줌
        * 예시: 주방장이 음식을 오븐에 넣어두고, 오븐이 조리하는 동안 다른 음식을 준비

* 필요성

    * **응답성 유지**
      * 특히 웹 서버나 UI가 있는 애플리케이션에서 LLM 호출처럼 시간이 오래 걸리는 작업을 비동기적으로 처리하면, 그 작업이 완료될 때까지 앱이 멈추지 않고 사용자에게 응답성을 유지 가능

    * **효율성 증가**
      *  여러 개의 LLM 호출이나 외부 API 호출을 동시에(병렬로) 시작하여 전체 처리 시간을 단축 가능 
      *  특히 네트워크 I/O 작업(데이터를 주고받는 작업)이 많은 경우에 효율적

* 비동기 코드를 실행하려면 파이썬의 `async`와 `await` 키워드를 사용해야 하며, 보통 **`async def`로 정의된 함수** 안에서 `await`를 사용 가능
* 주피터 노트북에서는 **셀 최상위 레벨**에서 **`await`를 직접 사용**

---

* **`astream`: 비동기 스트림** = **stream의 비동기 버전**

  * Q. 언제 쓸까?
    * A1. 웹 챗봇: 사용자가 답변을 기다리는 동안 UI가 멈추지 않고, 모델이 생성하는 텍스트가 실시간으로 사용자에게 보여져야 할 때 필수적
    * A2. 병렬 스트리밍: 여러 LLM 체인에서 동시에 스트리밍되는 결과를 받아 처리해야 할 때 유용
    * A3. 긴 텍스트 생성 중 응답성 유지: 모델이 아주 긴 답변을 생성하는 동안에도 다른 작업을 수행하거나 애플리케이션이 멈추지 않도록 하고 싶을 때

  * 특징
    * **`async for` 루프** 사용 -> 스트림의 각 조각을 비동기적으로 받음
    * `stream`과 마찬가지로 **조각별로 텍스트를 반환**하지만, **전체 과정이 비동기로 진행되어 메인 스레드를 막지 않음**
    * **`print(token, end="", flush=True)`** -> 즉시 출력 가능

In [ ]:
import asyncio                                                      # 비동기 코드 실행 위한 asyncio 모듈 임포트

# 이전 코드에서 chain 변수 (gemini-1.5-flash)
chain_astream = prompt | model2 | StrOutputParser()                 # stream -> gemini-2.5-flash-lite 모델로 새 체인 구성

# 스트리밍 출력을 위한 헬퍼 함수 (비동기 버전)
async def astream_and_print_response(answer_iterator):
    async for chunk in answer_iterator:                             # async for 사용
        print(chunk, end='', flush=True)                            
    print()                                                         # 스트림 완료 후 줄바꿈

print("\n--- astream: 비동기 스트림 (토픽: YouTube) ---")
try:
    # chain.astream 메서드를 사용하여 'YouTube' 토픽의 메시지를 비동기적으로 처리
    # astream은 비동기 이터레이터를 반환하므로, await와 async for를 사용
    await astream_and_print_response(chain_astream.astream({"topic": "YouTube"}))
    print("✅ astream 호출 성공.")
    print("---")
    
except Exception as e:
    print(f"❌ astream 호출 중 오류 발생: {e}")
    print("  -> 네트워크, API 키, 또는 모델 설정을 확인하세요.")
    print("---")

<small>

* 
    * 셀 출력

        * --- astream: 비동기 스트림 (토픽: YouTube) ---
        * YouTube는 전 세계 사용자가 비디오를 업로드, 시청 및 공유할 수 있는 온라인 비디오 공유 플랫폼입니다.  다양한 콘텐츠, 뮤직비디오부터 교육 영상, 게임 플레이까지 광범위하게 제공하며, 개인 크리에이터부터 대기업까지 다양한 채널이 존재합니다.  광고 수익 및 유료 멤버십 등 다양한 수익 모델을 통해 운영되고 있습니다.

        * ✅ astream 호출 성공.
        * ---

* **`ainvoke`: 비동기 호출** = **invoke의 비동기 버전**

  * Q.언제 쓸까?
    * A1. 단일 LLM 호출이 다른 비동기 작업과 함께 실행될 때: LLM 호출 결과가 나올 때까지 다른 비동기 작업을 기다리게 하고 싶지 않을 때 사용합니다.
    * A2. 웹 요청 처리: 웹 서버에서 사용자 요청을 처리하는 동안 LLM 호출이 백그라운드에서 비동기적으로 완료되도록 하여 서버의 응답성을 유지하고 싶을 때.

  * 특징
    * **`await` 키워드 사용** -> 비동기 함수가 완료될 때까지 기다림
    * **모든 응답이 완성된 후에 반환** = **스트리밍되지 않음** = **`invoke`**

In [ ]:
# 기본 체인 사용 (model = gemini-1.5-flash)

print("\n--- ainvoke: 비동기 호출 (토픽: NVDA) ---")
try:
    # chain 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 비동기적으로 처리합니다.
    # await 키워드를 사용하여 비동기 작업이 완료될 때까지 기다립니다.
    res_ainvoke = await chain.ainvoke({"topic": "NVDA"})
    print(res_ainvoke)
    print("✅ ainvoke 호출 성공.")
    print("---")
    
except Exception as e:
    print(f"❌ ainvoke 호출 중 오류 발생: {e}")
    print("  -> 네트워크, API 키, 또는 모델 설정을 확인하세요.")
    print("---")

<small>

* 
  * 셀 출력
      * --- ainvoke: 비동기 호출 (토픽: NVDA) ---
      * NVDA는 엔비디아(NVIDIA)의 주식 티커 심볼입니다. 엔비디아는 그래픽 처리 장치(GPU)를 설계하고 제조하는 미국의 반도체 회사입니다.  NVDA 주식은 기술 부문에서 가장 중요한 주식 중 하나로 간주되며, 인공지능, 게임, 데이터 센터 등 다양한 분야에서 사용되는 GPU의 성장에 따라 가치가 크게 변동합니다.
      * ✅ ainvoke 호출 성공.
      * ---

* **`abatch`: 비동기 배치** = **`batch`의 비동기 버전**

  * Q.언제 쓸까?
    * A1. **대량의 비동기 작업 동시 처리**: 수십, 수백 개의 LLM 호출을 한 번에 비동기적으로 시작하여 전체 처리 시간을 최소화하고 싶을 때
    * A2. **데이터 파이프라인의 병렬화**: 여러 데이터 포인트를 LLM을 통해 처리해야 하는데, 이 과정이 다른 비동기 구성 요소와 통합되어야 할 때

  * 특징
    * **`await` 키워드 사용** -> 모든 일괄 처리가 완료될 때까지 기다림
    * 입력 리스트에 대한 결과 리스트를 반환 = **`batch`**

In [ ]:
# 결과를 한 번에 반환 -> `batch` 동기식 처럼 시각적으로 확인은 어려움 
# 따라서 시간을 측정해서 비동기식이 빠르고 효율적이라는 것을 확인해보고자 함

import time
import asyncio

print("\n--- abatch: 비동기 배치 (토픽: YouTube, Instagram, Facebook) ---")

start_time = time.time()                                                        # 시작 시간 기록

# 기존의 chain 활용 (model = gemini-1.5-flash)

try:
    # chain 객체의 'abatch' 메서드를 사용하여 여러 토픽을 비동기적으로 일괄 처리
    batch_async_results = await chain.abatch(
        [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
    )
    
    end_time = time.time()                                                      # 종료 시간 기록
    
    for i, res in enumerate(batch_async_results):
        print(f"[{i+1}] {res}")
    print(f"✅ abatch 호출 성공. 소요 시간: {end_time - start_time:.2f} 초")        # 소요 시간 출력
    print("---")

except Exception as e:
    print(f"❌ abatch 호출 중 오류 발생: {e}")
    print("  -> 네트워크, API 키, 또는 모델 설정을 확인하세요.")
    print("---")

<small>

* 
  * 셀 출력
    * --- abatch: 비동기 배치 (토픽: YouTube, Instagram, Facebook) ---
    * [1] YouTube는 전 세계 사람들이 비디오를 업로드, 시청 및 공유할 수 있는 온라인 비디오 공유 플랫폼입니다.  다양한 콘텐츠, 뮤직비디오부터 교육 영상, 게임 플레이까지 광범위하게 제공하며, 개인 크리에이터부터 대기업까지 다양한 사용자들이 활동합니다.  광고 수익 및 유료 구독 서비스를 통해 수익을 창출하는 구글 자회사입니다.
    * [2] Instagram은 사진과 비디오 공유를 위한 소셜 네트워킹 서비스입니다. 사용자는 사진과 비디오에 필터를 적용하고, 해시태그를 사용하여 게시물을 분류하고, 다른 사용자를 팔로우하여 콘텐츠를 볼 수 있습니다. 전 세계 수억 명의 사용자가 사진과 비디오를 공유하고 소통하는 플랫폼입니다.
    * [3] 페이스북은 전 세계 사람들이 연결될 수 있도록 설계된 소셜 네트워킹 서비스입니다. 사용자는 프로필을 만들고, 친구를 추가하고, 사진과 비디오를 공유하고, 메시지를 주고받고, 그룹에 참여할 수 있습니다. 페이스북은 광고, 데이터 분석 및 기타 서비스를 통해 수익을 창출합니다.
    * ✅ abatch 호출 성공. 소요 시간: 2.02 초
    * ---

* **`astream_log`: 중간 단계 비동기 스트림**
  * `astream_log`  = 최종 응답 + **체인 실행 중 발생하는 중간 단계(intermediate steps)**까지 **스트리밍**하여 반환
  * 특히 **복잡한 체인**(예시: 여러 단계를 거치는 에이전트)에서 디버깅하거나, UI에서 '모델이 지금 무엇을 하고 있는지'를 사용자에게 보여줄 때 매우 유용

  * Q. 언제 쓸까?
    * A1. **복잡한 체인 디버깅**: 체인의 각 단계에서 어떤 입력이 들어가고 어떤 출력이 나오는지 실시간으로 확인하고 싶을 때
    * A2. **고급 UI 피드백**: 챗봇에서 '지금 도구를 검색 중...', '답변을 요약 중...'과 같은 중간 상태 메시지를 사용자에게 보여주고 싶을 때.
    * A3. **에이전트 동작 분석**: 에이전트가 어떤 도구를 호출하고 어떤 결과를 받는지 단계별로 추적하고 싶을 때.

  * 특징
    * 반환되는 각 '청크' **!= 단순한 텍스트** = `LogEntry` 객체와 같은 **더 구조화된 데이터**
        * 최종 출력 + 중간 단계의 입력/출력 + 상태 변화 등의 정보 포함
    * **`async for`** -> 각 청크의 **구조**를 이해하고 **필요한 정보에 접근**해야 함

In [ ]:
# 현재 체인(chain)이 단순하기 때문에, astream_log의 중간 단계 출력이 명확하지 않을 수 있음
# 복잡한 체인이나 에이전트에서 더욱 빛을 발하는 메서드이므로 교재의 내용을 따르지 않고 `RunnableParallel` + `astream_log`으로 적용
# 필요한 모듈은 파일 초반에 모두 임포트해둠
"""
###########
# 필요한 모듈
###########

# LangChain 관련
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 비동기 메서드 관련
import asyncio

# 운영체제 관련 (이 파일에서는 환경변수 접근 목적)
import os 

# gemini-API 관련
from langchain_google_genai import ChatGoogleGenerativeAI
"""


# 1. 모델 초기화 (모델 변경 : gemini-2.5-flash-lite)
try:
    model_for_parallel = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite", 
        temperature=0.1,
        google_api_key=os.getenv("GOOGLE_API_KEY") 
    )
    print("✅ Google GenAI 모델 (gemini-2.5-flash-lite) 초기화 성공.")
    print("---")
    
except Exception as e:
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    print("---")


# 2. Chain_1
chain1_capital = (
    PromptTemplate.from_template("{country} 의 수도는 어디야? 짧게 한 문장으로 대답해 줘.")
    | model_for_parallel                                        # 새로 정의한 모델 사용
    | StrOutputParser()
)

# 3. Chain_2
chain2_area = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야? 숫자로만 대답해 줘.")
    | model_for_parallel                                        # 새로 정의한 모델 사용
    | StrOutputParser()
)

# 4. RunnableParallel을 사용하여 두 체인을 병렬로 묶는 복합 체인 생성
# 'capital'과 'area'라는 키로 각 체인의 결과를 받을 수 있습니다.
combined_chain = RunnableParallel(capital=chain1_capital, area=chain2_area)

print("✅ 병렬 체인 (combined_chain) 구성 완료.")
print("---")


# 5. astream_log를 사용하여 병렬 체인의 중간 단계 스트림 확인
print("\n--- astream_log: 병렬 체인 중간 단계 스트림 (국가: 대한민국) ---")
print("  (출력되는 'Log Chunk' 객체의 구조를 유심히 살펴보세요. 복잡해 보일 수 있습니다.)")
print("  (각 'Log Chunk'는 JSON과 유사한 형태의 로그 데이터를 담고 있습니다.)")
try:
    async for chunk in combined_chain.astream_log({"country": "대한민국"}):
        print(f"Log Chunk Type: {type(chunk)}")
        print(f"Log Chunk Value: {chunk}")
        print("--- Log Chunk End ---\n")

        if hasattr(chunk, 'ops') and chunk.ops:
            for op in chunk.ops:
                # dict 형태일 경우 예외 방지용 get 처리
                path = op.get("path", "")
                value = op.get("value", "")

                if path == "/logs/capital/streamed_output_str":
                    print(f"  [수도] 스트리밍 텍스트 부분: {value}", end='', flush=True)
                elif path == "/logs/area/streamed_output_str":
                    print(f"  [면적] 스트리밍 텍스트 부분: {value}", end='', flush=True)
            print("---")

    print("\n✅ astream_log 호출 성공 (병렬 체인).")
    print("--- 최종 답변 ---")
    final_result = await combined_chain.ainvoke({"country": "대한민국"})
    print(f"수도: {final_result['capital']}")
    print(f"면적: {final_result['area']}")
    print("---")

except Exception as e:
    print(f"❌ astream_log 호출 중 오류 발생: {e}")
    print("  -> 네트워크 연결, GOOGLE_API_KEY 환경 변수, 또는 모델 설정을 다시 확인해 보세요.")
    print("---")




<small>

* 실패 코드 및 에러 메시지
    * 5. astream_log를 사용하여 병렬 체인의 중간 단계 스트림 확인 - 실패 코드
        ```
        print("\n--- astream_log: 병렬 체인 중간 단계 스트림 (국가: 대한민국) ---")
        print("  (출력되는 'Log Chunk' 객체의 구조를 유심히 살펴보세요. 복잡해 보일 수 있습니다.)")
        print("  (각 'Log Chunk'는 JSON과 유사한 형태의 로그 데이터를 담고 있습니다.)")
        
        try:
            # combined_chain에 astream_log를 사용하여 중간 단계를 스트리밍합니다.
            async for chunk in combined_chain.astream_log({"country": "대한민국"}):
                print(f"Log Chunk Type: {type(chunk)}")                             # 청크의 타입 확인
                print(f"Log Chunk Value: {chunk}")                                  # Raw 청크 객체 전체 출력 (JSON과 유사한 구조)
                print("--- Log Chunk End ---\n")                                    # 각 청크 구분선 추가

                # LangChain 버전에 따라 LogEntry 객체의 접근 방식이 다를 수 있음
                # 일반적으로 `ops` 리스트를 통해 변경 사항을 확인
                if hasattr(chunk, 'ops') and chunk.ops:
                    for op in chunk.ops:
                        path = op.get("path", "")
                        value = op.get("value", "")
                        
                        # 진단용 디버깅 코드
                        print(f"[DEBUG] op 객체: {op}")
                        print(f"[DEBUG] -> type: {type(op)}")
                        
                        # `streamed_output_str` 경로로 최종 LLM 출력 조각을 찾기
                        # LLM이 실제 답변 텍스트를 스트리밍하는 부분
                        if op.path == "/logs/capital/streamed_output_str":
                            print(f"  [수도] 스트리밍 텍스트 부분: {op.value}", end='', flush=True)
                        elif op.path == "/logs/area/streamed_output_str":
                            print(f"  [면적] 스트리밍 텍스트 부분: {op.value}", end='', flush=True)
                    print("---")                                                     # 각 스트리밍 텍스트 부분 출력 후 구분선 추가 (선택 사항)

            print("\n✅ astream_log 호출 성공 (병렬 체인).")
            print("--- 최종 답변 ---")
            final_result = await combined_chain.ainvoke({"country": "대한민국"})       # 최종 결과 확인
            print(f"수도: {final_result['capital']}")
            print(f"면적: {final_result['area']}")
            print("---")

        except Exception as e:
            print(f"❌ astream_log 호출 중 오류 발생: {e}")
            print("  -> 네트워크 연결, GOOGLE_API_KEY 환경 변수, 또는 모델 설정을 다시 확인해 보세요.")
            print("---")
        ```
    * 
    * 에러 메시지
        * ✅ Google GenAI 모델 (gemini-2.5-flash-lite) 초기화 성공.
        * ---
        * ✅ 병렬 체인 (combined_chain) 구성 완료.
        * ---
        * 
        * --- astream_log: 병렬 체인 중간 단계 스트림 (국가: 대한민국) ---
        * (출력되는 'Log Chunk' 객체의 구조를 유심히 살펴보세요. 복잡해 보일 수 있습니다.)
        * (각 'Log Chunk'는 JSON과 유사한 형태의 로그 데이터를 담고 있습니다.)
        * Log Chunk Type: <class 'langchain_core.tracers.log_stream.RunLogPatch'>
        * Log Chunk Value: RunLogPatch({'op': 'replace',
        * 'path': '',
        * 'value': {'final_output': None,
        *             'id': '32574452-3b48-4a44-8dff-fec38f5db4b4',
        *             'logs': {},
        *             'name': 'RunnableParallel<capital,area>',
        *             'streamed_output': [],
        *             'type': 'chain'}})
        * --- Log Chunk End ---
        * 
        * [DEBUG] op 객체: {'op': 'replace', 'path': '', 'value': {'id': '32574452-3b48-4a44-8dff-fec38f5db4b4', 'streamed_output': [], 'final_output': None, 'logs': {}, 'name': 'RunnableParallel<capital,area>', 'type': 'chain'}}
        * [DEBUG] -> type: <class 'dict'>
        * ❌ astream_log 호출 중 오류 발생: 'dict' object has no attribute 'path' -> 네트워크 연결, GOOGLE_API_KEY 환경 변수, 또는 모델 설정을 다시 확인해 보세요.
        * ---
    * 
    * 에러 메시지 해석
        * `chunk.ops` 안의 각 요소가 `PatchEntry` 객체가 아니라 단순 `dict`로 들어오고 있음
        * LangChain이 내부적으로 사용하는 `astream_log()`의 구조가 JSON-like `dict` 형태로 반환 -> 기존의 `.path`나 `.value` 속성에 점 표기법 접근이 불가능
    * 
    * 해결 방법 : **딕셔너리 기반**으로 수정 -> `get()`을 활용한 방식으로 코드 재작성 

----

<small>

* 
  * 셀 출력
  
  ---

      * ✅ Google GenAI 모델 (gemini-2.5-flash-lite) 초기화 성공.

    ---

      * ✅ 병렬 체인 (combined_chain) 구성 완료.

    ---

      * --- astream_log: 병렬 체인 중간 단계 스트림 (국가: 대한민국) ---
      * (출력되는 'Log Chunk' 객체의 구조를 유심히 살펴보세요. 복잡해 보일 수 있습니다.)
      * (각 'Log Chunk'는 JSON과 유사한 형태의 로그 데이터를 담고 있습니다.)
      * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
      * Log Chunk Value: ```RunLogPatch({'op': 'replace',
      'path': '',
      'value': {'final_output': None,
                  'id': 'f3c1dbc5-6906-4740-a9d9-6732d34c2041',
                  'logs': {},
                  'name': 'RunnableParallel<capital,area>',
                  'streamed_output': [],
                  'type': 'chain'}})```
      * --- Log Chunk End ---

    ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
    'path': '/logs/RunnableSequence',
    'value': {'end_time': None,
                'final_output': None,
                'id': '0dd23fed-1e08-468d-9a76-afb4d89691ec',
                'metadata': {},
                'name': 'RunnableSequence',
                'start_time': '2025-07-30T03:27:49.954+00:00',
                'streamed_output': [],
                'streamed_output_str': [],
                'tags': ['map:key:capital'],
                'type': 'chain'}})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/RunnableSequence:2',
      'value': {'end_time': None,
                'final_output': None,
                'id': 'd3a4db63-4c19-411e-b9d5-7de81352cfbf',
                'metadata': {},
                'name': 'RunnableSequence',
                'start_time': '2025-07-30T03:27:49.955+00:00',
                'streamed_output': [],
                'streamed_output_str': [],
                'tags': ['map:key:area'],
                'type': 'chain'}})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/PromptTemplate',
      'value': {'end_time': None,
                'final_output': None,
                'id': '97009445-8949-49ba-af3f-6ef62902e835',
                'metadata': {},
                'name': 'PromptTemplate',
                'start_time': '2025-07-30T03:27:49.956+00:00',
                'streamed_output': [],
                'streamed_output_str': [],
                'tags': ['seq:step:1'],
                'type': 'prompt'}})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/PromptTemplate:2',
      'value': {'end_time': None,
                'final_output': None,
                'id': 'd03bef57-5ff7-42a8-b0f3-1f4027efb9d7',
                'metadata': {},
                'name': 'PromptTemplate',
                'start_time': '2025-07-30T03:27:49.957+00:00',
                'streamed_output': [],
                'streamed_output_str': [],
                'tags': ['seq:step:1'],
                'type': 'prompt'}})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/PromptTemplate/final_output',
      'value': StringPromptValue(text='대한민국 의 면적은 얼마야? 숫자로만 대답해 줘.')},
    {'op': 'add',
      'path': '/logs/PromptTemplate/end_time',
      'value': '2025-07-30T03:27:49.957+00:00'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/PromptTemplate:2/final_output',
      'value': StringPromptValue(text='대한민국 의 수도는 어디야? 짧게 한 문장으로 대답해 줘.')},
    {'op': 'add',
      'path': '/logs/PromptTemplate:2/end_time',
      'value': '2025-07-30T03:27:49.958+00:00'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI',
      'value': {'end_time': None,
                'final_output': None,
                'id': '5058be8f-2c75-4ac6-9b0d-874c4db2dbf6',
                'metadata': {'ls_model_name': 'gemini-2.5-flash-lite',
                            'ls_model_type': 'chat',
                            'ls_provider': 'google_genai',
                            'ls_temperature': 0.1},
                'name': 'ChatGoogleGenerativeAI',
                'start_time': '2025-07-30T03:27:49.961+00:00',
                'streamed_output': [],
                'streamed_output_str': [],
                'tags': ['seq:step:2'],
                'type': 'llm'}})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI:2',
      'value': {'end_time': None,
                'final_output': None,
                'id': '7030e4c1-7539-42ce-93c1-936dd281ba39',
                'metadata': {'ls_model_name': 'gemini-2.5-flash-lite',
                            'ls_model_type': 'chat',
                            'ls_provider': 'google_genai',
                            'ls_temperature': 0.1},
                'name': 'ChatGoogleGenerativeAI',
                'start_time': '2025-07-30T03:27:49.961+00:00',
                'streamed_output': [],
                'streamed_output_str': [],
                'tags': ['seq:step:2'],
                'type': 'llm'}})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI:2/streamed_output_str/-',
      'value': '대한민'},
    {'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI:2/streamed_output/-',
      'value': AIMessageChunk(content='대한민', additional_kwargs={}, response_metadata={'safety_ratings': []}, id='run--7030e4c1-7539-42ce-93c1-936dd281ba39', usage_metadata={'input_tokens': 21, 'output_tokens': 2, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}})})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/StrOutputParser',
      'value': {'end_time': None,
                'final_output': None,
                'id': '1a8f28e1-efce-44de-a362-d1a7ec36e30a',
                'metadata': {},
                'name': 'StrOutputParser',
                'start_time': '2025-07-30T03:27:51.306+00:00',
                'streamed_output': [],
                'streamed_output_str': [],
                'tags': ['seq:step:3'],
                'type': 'parser'}})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/StrOutputParser/streamed_output/-',
      'value': '대한민'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/RunnableSequence/streamed_output/-',
      'value': '대한민'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': {'capital': '대한민'}},
    {'op': 'replace', 'path': '/final_output', 'value': {'capital': '대한민'}})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI:2/streamed_output_str/-',
      'value': '국의 수도는 서울입니다.'},
    {'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI:2/streamed_output/-',
      'value': AIMessageChunk(content='국의 수도는 서울입니다.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--7030e4c1-7539-42ce-93c1-936dd281ba39', usage_metadata={'input_tokens': 0, 'output_tokens': 6, 'total_tokens': 6, 'input_token_details': {'cache_read': 0}})})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/StrOutputParser/streamed_output/-',
      'value': '국의 수도는 서울입니다.'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```unLogPatch({'op': 'add',
      'path': '/logs/RunnableSequence/streamed_output/-',
      'value': '국의 수도는 서울입니다.'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/streamed_output/-',
      'value': {'capital': '국의 수도는 서울입니다.'}},
    {'op': 'replace',
      'path': '/final_output/capital',
      'value': '대한민국의 수도는 서울입니다.'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI:2/final_output',
      'value': {'generations': [[{'generation_info': {'finish_reason': 'STOP',
                                                      'model_name': 'gemini-2.5-flash-lite',
                                                      'safety_ratings': []},
                                  'message': AIMessageChunk(content='대한민국의 수도는 서울입니다.', additional_kwargs={}, response_metadata={'safety_ratings': [], 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite'}, id='run--7030e4c1-7539-42ce-93c1-936dd281ba39', usage_metadata={'input_tokens': 21, 'output_tokens': 8, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}}),
                                  'text': '대한민국의 수도는 서울입니다.',
                                  'type': 'ChatGenerationChunk'}]],
                'llm_output': None,
                'run': None,
                'type': 'LLMResult'}},
    {'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI:2/end_time',
      'value': '2025-07-30T03:27:51.404+00:00'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/StrOutputParser/final_output',
      'value': {'output': '대한민국의 수도는 서울입니다.'}},
    {'op': 'add',
      'path': '/logs/StrOutputParser/end_time',
      'value': '2025-07-30T03:27:51.404+00:00'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/RunnableSequence/final_output',
      'value': {'output': '대한민국의 수도는 서울입니다.'}},
    {'op': 'add',
      'path': '/logs/RunnableSequence/end_time',
      'value': '2025-07-30T03:27:51.405+00:00'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI/streamed_output_str/-',
      'value': '100'},
    {'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI/streamed_output/-',
      'value': AIMessageChunk(content='100', additional_kwargs={}, response_metadata={'safety_ratings': []}, id='run--5058be8f-2c75-4ac6-9b0d-874c4db2dbf6', usage_metadata={'input_tokens': 19, 'output_tokens': 3, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}})})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/StrOutputParser:2',
      'value': {'end_time': None,
                'final_output': None,
                'id': '9852fb30-118e-407f-aaa5-a942bc1c951e',
                'metadata': {},
                'name': 'StrOutputParser',
                'start_time': '2025-07-30T03:27:51.982+00:00',
                'streamed_output': [],
                'streamed_output_str': [],
                'tags': ['seq:step:3'],
                'type': 'parser'}})``
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/StrOutputParser:2/streamed_output/-',
      'value': '100'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/RunnableSequence:2/streamed_output/-',
      'value': '100'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: `RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': {'area': '100'}},
    {'op': 'add', 'path': '/final_output/area', 'value': '100'})`
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI/streamed_output_str/-',
      'value': '428'},
    {'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI/streamed_output/-',
      'value': AIMessageChunk(content='428', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--5058be8f-2c75-4ac6-9b0d-874c4db2dbf6', usage_metadata={'input_tokens': 0, 'output_tokens': 3, 'total_tokens': 3, 'input_token_details': {'cache_read': 0}})})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/StrOutputParser:2/streamed_output/-',
      'value': '428'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/RunnableSequence:2/streamed_output/-',
      'value': '428'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': {'area': '428'}},
    {'op': 'replace', 'path': '/final_output/area', 'value': '100428'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI/final_output',
      'value': {'generations': [[{'generation_info': {'finish_reason': 'STOP',
                                                      'model_name': 'gemini-2.5-flash-lite',
                                                      'safety_ratings': []},
                                  'message': AIMessageChunk(content='100428', additional_kwargs={}, response_metadata={'safety_ratings': [], 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite'}, id='run--5058be8f-2c75-4ac6-9b0d-874c4db2dbf6', usage_metadata={'input_tokens': 19, 'output_tokens': 6, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}}),
                                  'text': '100428',
                                  'type': 'ChatGenerationChunk'}]],
                'llm_output': None,
                'run': None,
                'type': 'LLMResult'}},
    {'op': 'add',
      'path': '/logs/ChatGoogleGenerativeAI/end_time',
      'value': '2025-07-30T03:27:51.985+00:00'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/StrOutputParser:2/final_output',
      'value': {'output': '100428'}},
    {'op': 'add',
      'path': '/logs/StrOutputParser:2/end_time',
      'value': '2025-07-30T03:27:51.985+00:00'})```
    * --- Log Chunk End ---

      ---

    * Log Chunk Type: `<class 'langchain_core.tracers.log_stream.RunLogPatch'>`
    * Log Chunk Value: ```RunLogPatch({'op': 'add',
      'path': '/logs/RunnableSequence:2/final_output',
      'value': {'output': '100428'}},
    {'op': 'add',
      'path': '/logs/RunnableSequence:2/end_time',
      'value': '2025-07-30T03:27:51.986+00:00'})```
    * --- Log Chunk End ---

    ---

    * ✅ astream_log 호출 성공 (병렬 체인).
    * --- 최종 답변 ---
    * 수도: 대한민국의 수도는 서울입니다.
    * 면적: 100428
    * ---